In [1]:
import numpy as np
import os
import sys


# Navigate to the parent directory of the project structure
project_dir = os.path.abspath(os.path.join(os.getcwd(), '../..'))
src_dir = os.path.join(project_dir, 'src')
log_dir = os.path.join(project_dir, 'log')
fig_dir = os.path.join(project_dir, 'fig')

# Add the src directory to sys.path
sys.path.append(src_dir)

from estimator.NDIS import BasicNDISEstimator
from analysis.archive_NDIS import _check_kd_asymptotic_dist_delta
from accounting.Gaussian_shifted_covariance import delta_gaussian_1d_closed_form, delta_gaussian_1d_closed_form_reverse

In [9]:
mu_1 = np.array([0])
mu_2 = np.array([1])
Sigma_1 = np.array([[10]])
Sigma_2 = np.array([[1]])
eps = 2

estimator = BasicNDISEstimator(mu_1, Sigma_1, mu_2, Sigma_2, workers=50, num_samples=2**16)

print(f"Estimated NDIS: {estimator.estimate(eps)} with epsilon = {eps}")

estimator_1 = BasicNDISEstimator(mu_1, Sigma_1, mu_1, Sigma_2, workers=50, num_samples=2**16)
delta_1 = estimator_1.estimate(eps)["delta"]

estimator_2 = BasicNDISEstimator(mu_1, Sigma_2, mu_2, Sigma_2, workers=50, num_samples=2**16)
delta_2 = estimator_2.estimate(eps)["delta"]

print(f"Estimated NDIS for triangle: {delta_1 + delta_2} with epsilon = {eps}")

Estimated NDIS: {'delta': np.float64(0.3681177241625452)} with epsilon = 2
Estimated NDIS for triangle: 0.3634794060966442 with epsilon = 2


In [2]:
import numpy as np

# --- YOUR closed forms ---
# delta_gaussian_1d_closed_form(mu, sigma1, sigma2, eps)          # forward
# delta_gaussian_1d_closed_form_reverse(mu, sigma1, sigma2, eps)   # reverse

# --- Estimator factories ---
# Forward: X~N(0,σ1^2), Y~N(μ,σ2^2)
estimator_factory_fwd = lambda mu, sigma1, sigma2: BasicNDISEstimator(
    np.array([0.0]),        np.array([[sigma1**2]]),
    np.array([mu]),         np.array([[sigma2**2]]),
    workers=50, num_samples=10**7
)

# Reverse: X_rev~N(μ,σ2^2), Y_rev~N(0,σ1^2)
estimator_factory_rev = lambda mu, sigma1, sigma2: BasicNDISEstimator(
    np.array([mu]),         np.array([[sigma2**2]]),
    np.array([0.0]),        np.array([[sigma1**2]]),
    workers=50, num_samples=10**7
)

def _delta_from_estimator(factory, mu, sigma1, sigma2, eps):
    est = factory(mu, sigma1, sigma2)
    return float(est.estimate(eps)["delta"])

def _eps_grid(mu, sigma1, sigma2):
    # Use the same grid for both directions; it doesn’t need to be perfect.
    pivot = (mu*mu)/(2.0*sigma2) - 0.5*np.log(sigma1/sigma2)
    base = [0.0, 0.02, 0.05, 0.1, 0.2, 0.5, 1.0, 2.0, 3.0]
    around = [max(0.0, round(pivot + off, 6)) for off in (-0.2, -0.05, 0.0, 0.05, 0.2)]
    return sorted(set(base + around))

def run_closed_form_vs_estimator_tests_bidir(
    sigma2_list=(0.25, 0.5, 1.0, 2.0, 3.0),
    gamma_list=(1.05, 1.1, 1.25, 1.5, 2.0, 3.0, 5.0),
    mu_scales=(0.0, 0.05, 0.1, 0.25, 0.5, 1.0, 2.0),
    atol=5e-3, rtol=1e-2
):
    print("dir\tmu\tsigma1\tsigma2\teps\test\tcf\tabs_err\trel_err\tresult")
    total=passed=failed=errors=0

    for sigma2 in sigma2_list:
        for g in gamma_list:
            sigma1 = sigma2 * g   # σ1 > σ2 (stddevs)
            mus = sorted(set([m*np.sqrt(sigma2) for m in mu_scales] +
                             [-m*np.sqrt(sigma2) for m in mu_scales]))
            for mu in mus:
                for eps in _eps_grid(mu, sigma1, sigma2):

                    # ----- forward δ_{X,Y} -----
                    total += 1
                    try:
                        cf_fwd  = float(delta_gaussian_1d_closed_form(mu, sigma1, sigma2, eps))
                        est_fwd = _delta_from_estimator(estimator_factory_fwd, mu, sigma1, sigma2, eps)
                        abs_err = abs(est_fwd - cf_fwd)
                        rel_err = abs_err / max(1.0, abs(cf_fwd))
                        ok = abs_err <= max(atol, rtol*max(1.0, abs(cf_fwd)))
                        print(f"FWD\t{mu:.6g}\t{sigma1:.6g}\t{sigma2:.6g}\t{eps:.6g}\t"
                              f"{est_fwd:.6g}\t{cf_fwd:.6g}\t{abs_err:.3g}\t{rel_err:.3g}\t"
                              f"{'PASS' if ok else 'FAIL'}")
                        passed += int(ok); failed += int(not ok)
                    except Exception as e:
                        errors += 1
                        print(f"FWD\t{mu:.6g}\t{sigma1:.6g}\t{sigma2:.6g}\t{eps:.6g}\t"
                              f"ERR\tERR\t-\t-\tERROR: {type(e).__name__}: {e}")

                    # ----- reverse δ_{Y,X} (means and sigmas swapped!) -----
                    total += 1
                    try:
                        cf_rev  = float(delta_gaussian_1d_closed_form_reverse(mu, sigma1, sigma2, eps))
                        est_rev = _delta_from_estimator(estimator_factory_rev, mu, sigma1, sigma2, eps)
                        abs_err = abs(est_rev - cf_rev)
                        rel_err = abs_err / max(1.0, abs(cf_rev))
                        ok = abs_err <= max(atol, rtol*max(1.0, abs(cf_rev)))
                        print(f"REV\t{mu:.6g}\t{sigma1:.6g}\t{sigma2:.6g}\t{eps:.6g}\t"
                              f"{est_rev:.6g}\t{cf_rev:.6g}\t{abs_err:.3g}\t{rel_err:.3g}\t"
                              f"{'PASS' if ok else 'FAIL'}")
                        passed += int(ok); failed += int(not ok)
                    except Exception as e:
                        errors += 1
                        print(f"REV\t{mu:.6g}\t{sigma1:.6g}\t{sigma2:.6g}\t{eps:.6g}\t"
                              f"ERR\tERR\t-\t-\tERROR: {type(e).__name__}: {e}")
                    
                    assert cf_fwd >= cf_rev

    print("\nSummary:")
    print(f"  Total:  {total}")
    print(f"  Passed: {passed}")
    print(f"  Failed: {failed}")
    print(f"  Errors: {errors}")


In [4]:
run_closed_form_vs_estimator_tests_bidir()

dir	mu	sigma1	sigma2	eps	est	cf	abs_err	rel_err	result
FWD	-1	0.2625	0.25	0	0.948987	0.948987	1.11e-16	1.11e-16	PASS
REV	-1	0.2625	0.25	0	0.948987	0.948987	1.11e-16	1.11e-16	PASS
FWD	-1	0.2625	0.25	0.02	0.94849	0.94849	0	0	PASS
REV	-1	0.2625	0.25	0.02	0.94846	0.94846	1.11e-16	1.11e-16	PASS
FWD	-1	0.2625	0.25	0.05	0.947737	0.947737	1.11e-16	1.11e-16	PASS
REV	-1	0.2625	0.25	0.05	0.947661	0.947661	0	0	PASS
FWD	-1	0.2625	0.25	0.1	0.946463	0.946463	4.44e-16	4.44e-16	PASS
REV	-1	0.2625	0.25	0.1	0.946307	0.946307	3.33e-16	3.33e-16	PASS
FWD	-1	0.2625	0.25	0.2	0.94384	0.94384	1.11e-16	1.11e-16	PASS
REV	-1	0.2625	0.25	0.2	0.943513	0.943513	2.22e-16	2.22e-16	PASS
FWD	-1	0.2625	0.25	0.5	0.935365	0.935365	3.33e-16	3.33e-16	PASS
REV	-1	0.2625	0.25	0.5	0.934419	0.934419	3.33e-16	3.33e-16	PASS
FWD	-1	0.2625	0.25	1	0.919084	0.919084	2.22e-16	2.22e-16	PASS
REV	-1	0.2625	0.25	1	0.916713	0.916713	1.11e-16	1.11e-16	PASS
FWD	-1	0.2625	0.25	1.77561	0.888005	0.888005	5.55e-16	5.55e-16	PASS
REV	-1	0.2625	0.25	